In [1]:
from loguru import logger


from slt_positional_bias.dataset import generate_data_frame, generate_merged_data_frame

df_docs = generate_data_frame("data/interim/inputs/corpus.jsonl")
df_topics = generate_data_frame("data/interim/inputs/queries.jsonl")
df_qrels = generate_data_frame("data/interim/inputs/qrels.rag24.test-umbrela-all.txt")

df_qrels.columns = ["topic_id", "q0", "doc_id", "rel_scoring"]

df_topics = df_topics.rename(columns={"qid": "topic_id","query": "topic"})
df_docs = df_docs.rename(columns={"docno": "doc_id", "text": "doc"})

logger.info(f"df_qrels column 'topic_id' is unique: {df_qrels["topic_id"].is_unique}")
logger.info(f"df_topics column 'topic_id' is unique: {df_topics["topic_id"].is_unique}")
logger.info(f"df_qrels column 'doc_id' is unique: {df_qrels["doc_id"].is_unique}")
logger.info(f"df_docs column 'doc_id' is unique: {df_docs["doc_id"].is_unique}")
logger.info("--------------------------------------")

df_merged = (
    df_qrels
    .loc[:, ["topic_id", "doc_id", "rel_scoring"]]
    .merge(
        df_topics.loc[:, ["topic_id", "topic"]],
        on="topic_id",
        how="inner"
    )
    .merge(
        df_docs.loc[:, ["doc_id", "doc"]],
        on="doc_id",
        how="inner"
    )
)

df_merged

2025-07-22 16:14:05.579 | INFO     | slt_positional_bias.config:<module>:11 - PROJ_ROOT path is: C:\Users\Albert\Documents\SLT\slt_group_2_positional_bias


2025-07-22 16:14:07.712 | INFO     | slt_positional_bias.dataset:generate_data_frame:50 - file path exists
2025-07-22 16:14:15.046 | INFO     | slt_positional_bias.dataset:generate_data_frame:50 - file path exists
2025-07-22 16:14:15.068 | INFO     | slt_positional_bias.dataset:generate_data_frame:50 - file path exists
2025-07-22 16:14:15.284 | INFO     | __main__:<module>:15 - df_qrels column 'topic_id' is unique: False
2025-07-22 16:14:15.285 | INFO     | __main__:<module>:16 - df_topics column 'topic_id' is unique: True
2025-07-22 16:14:15.316 | INFO     | __main__:<module>:17 - df_qrels column 'doc_id' is unique: False
2025-07-22 16:14:15.368 | INFO     | __main__:<module>:18 - df_docs column 'doc_id' is unique: True
2025-07-22 16:14:15.369 | INFO     | __main__:<module>:19 - --------------------------------------


,topic_id,doc_id,rel_scoring,topic,doc
0,2024-145979,msmarco_v2.1_doc_01_523681915#0_449763684,2,what is vicarious trauma and how can it be cop...,Vicarious traumatization - Wikipedia Vicarious...
1,2024-145979,msmarco_v2.1_doc_13_1647729865#0_3617397938,3,what is vicarious trauma and how can it be cop...,Vicarious Trauma: What Causes it and Ways to H...
2,2024-145979,msmarco_v2.1_doc_25_1148700328#0_2179512504,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? | What is Vicarious ...
3,2024-145979,msmarco_v2.1_doc_25_1148700328#1_2179514079,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? | What is Vicarious ...
4,2024-145979,msmarco_v2.1_doc_44_1043805224#2_2182641075,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? - Jefferson Center -...
...,...,...,...,...,...
29158,2024-127266,msmarco_v2.1_doc_58_1230815815#5_1795053441,2,what are some key challenges related to the re...,Highest E-Waste Generating Nations In The Worl...
29159,2024-127266,msmarco_v2.1_doc_34_470264225#3_1018882699,0,what are some key challenges related to the re...,The Pros & Cons Of Landfills (Benefits & Disad...
29160,2024-127266,msmarco_v2.1_doc_57_781229369#6_1221950031,1,what are some key challenges related to the re...,The Patron Saint of Electronics? Do you Know t...
29161,2024-127266,msmarco_v2.1_doc_57_781235684#6_1221962946,1,what are some key challenges related to the re...,Destroy VHS Tapes \nDestroy VHS Tapes\nDaliah ...


In [2]:
df_merged_left = generate_merged_data_frame()
df_merged_left

2025-07-04 15:48:42.836 | INFO     | slt_positional_bias.dataset:generate_data_frame:61 - file path exists
2025-07-04 15:48:52.229 | INFO     | slt_positional_bias.dataset:generate_data_frame:61 - file path exists
2025-07-04 15:48:52.236 | INFO     | slt_positional_bias.dataset:generate_data_frame:61 - file path exists
2025-07-04 15:48:52.541 | INFO     | slt_positional_bias.dataset:generate_merged_data_frame:23 - df_qrels column 'topic_id' is unique: False
2025-07-04 15:48:52.542 | INFO     | slt_positional_bias.dataset:generate_merged_data_frame:24 - df_topics column 'topic_id' is unique: True
2025-07-04 15:48:52.574 | INFO     | slt_positional_bias.dataset:generate_merged_data_frame:25 - df_qrels column 'doc_id' is unique: False
2025-07-04 15:48:52.623 | INFO     | slt_positional_bias.dataset:generate_merged_data_frame:26 - df_docs column 'doc_id' is unique: True
2025-07-04 15:48:52.623 | INFO     | slt_positional_bias.dataset:generate_merged_data_frame:27 - ------------------------

,topic_id,doc_id,rel_scoring,topic,doc
0,2024-145979,msmarco_v2.1_doc_01_523681915#0_449763684,2,what is vicarious trauma and how can it be cop...,Vicarious traumatization - Wikipedia Vicarious...
1,2024-145979,msmarco_v2.1_doc_13_1647729865#0_3617397938,3,what is vicarious trauma and how can it be cop...,Vicarious Trauma: What Causes it and Ways to H...
2,2024-145979,msmarco_v2.1_doc_25_1148700328#0_2179512504,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? | What is Vicarious ...
3,2024-145979,msmarco_v2.1_doc_25_1148700328#1_2179514079,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? | What is Vicarious ...
4,2024-145979,msmarco_v2.1_doc_44_1043805224#2_2182641075,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? - Jefferson Center -...
...,...,...,...,...,...
108473,2024-21669,msmarco_v2.1_doc_08_298503251#1_568833050,0,NaN,NaN
108474,2024-21669,msmarco_v2.1_doc_27_877180220#2_1765802089,1,NaN,NaN
108475,2024-21669,msmarco_v2.1_doc_01_1752638978#7_2584141250,1,NaN,NaN
108476,2024-21669,msmarco_v2.1_doc_09_914336366#11_1554208777,0,NaN,NaN
